In [1]:
import os 
from dataclasses import dataclass




In [2]:
%pwd
os.chdir("../")

In [3]:
%pwd

'd:\\Projects\\Alzeimer\\Alzheimer-s-Disease-Detection'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
# artifacts entity 
@dataclass
class DataIngestionArtifact:
    train_path:str
    test_path:str
    

In [10]:
import os
from dataclasses import dataclass
from datetime import datetime
from alzheimer_disease.constant.training_pipeline import *



TIMESTAMP: str = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")

@dataclass
class TrainingPipelineConfig:
    artifacts_dir: str = os.path.join(ARTIFACTS_DIR,TIMESTAMP)

training_pipeline_config:TrainingPipelineConfig = TrainingPipelineConfig() 




@dataclass
class DataIngestionConfig:
    data_ingestion_dir: str = os.path.join(
        training_pipeline_config.artifacts_dir, DATA_INGESTION_DIR_NAME
    )

    feature_store_file_path: str = os.path.join(
        data_ingestion_dir, DATA_INGESTION_FEATURE_STORE_DIR
    )

    database_name : str = DATA_BASE_NAME
    train_collection_name : str = TRAIN_COLLECTION_NAME
    test_collection_name : str = TEST_COLLECTION_NAME
    
    

    





  


In [12]:
from alzheimer_disease.exception import AlzException
from alzheimer_disease.utils.main_utils import get_data_from_mongodb
from alzheimer_disease.logger import logging
import sys

class DataIngestion:
    
    def __init__(self,data_ingestion_config:DataIngestionConfig ):
        try:
            logging.info(f"{'>>'*20} Data Ingestion {'<<'*20}")
            self.data_ingestion_config = data_ingestion_config
        except Exception as e:
            raise AlzException(e, sys)

    def initiate_data_ingestion(self)->DataIngestionArtifact:
        try:
            logging.info(f"Collecting data from mongodb...")

            feature_store_dir = os.path.dirname(self.data_ingestion_config.feature_store_file_path)
            logging.info("Create feature store folder if not available")
            os.makedirs(feature_store_dir,exist_ok=True)

            logging.info("Save data to feature store folder")
            train_path, test_path = get_data_from_mongodb(output_folder_path=feature_store_dir,
                                  database_name=self.data_ingestion_config.database_name,
                                  train_collection_name=self.data_ingestion_config.train_collection_name,
                                  test_collection_name=self.data_ingestion_config.test_collection_name)



           
            data_ingestion_artifact = DataIngestionArtifact(train_path=train_path , test_path=test_path)
            
            logging.info(f"Data ingestion artifact: {data_ingestion_artifact}")
            return data_ingestion_artifact

        except Exception as e:
            raise AlzException(e,sys)

In [13]:

try :
    #data ingestion
    training_pipeline_config = TrainingPipelineConfig()
    data_ingestion_config  = DataIngestionConfig()

    # Usage example

    data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)
    data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

except Exception as e:
    raise AlzException(e,sys)
